In [1]:
import requests, time
from bs4 import BeautifulSoup
import threading as thrd


def Department_list():
    # get the main page of class-signing
    url = 'http://course-query.acad.ncku.edu.tw/qry'
    soup = get_site(url)
    # print(soup)

    #create a dictionary in the form of {department_No. : site_link} for each department
    dept_all = soup.find('ul',id='dept_list')
    dept = dept_all.find_all('div', 'dept')
    dept_link = []
    for i in dept:
        dept_link.append("http://course-query.acad.ncku.edu.tw/qry/" + i.find('a').get('href'))
    dept_text = []
    for i in dept:
        dept_text.append(i.string)
    dept_no = []
    for string in dept_text:
        dept_no.append(string[string.find('(')+2 : string.find(u' ）')])
    dept_dict = dict(zip(dept_no, dept_link))
    # print(dept_dict)
    
    return dept_dict

In [2]:
def Course_list(DeptNo, dept_dict):
    # create a dictionary in the form of {course_No. : [course_name, Space_Available]}
    soup = get_site(dept_dict[DeptNo])
    
    course_tmp = []
    course_all = []
    course_tmp = soup.find_all('td')
    for row in range(1,int(len(course_tmp)/24)+1):
        course_all.append(course_tmp[24*(row-1):24*row-1])
    course_dict = {}
    for i in course_all:
        if i[2].string == None:
            continue
        course_dict.update({i[2].string: [i[10].string, i[15].string]})
        
    return course_dict

In [3]:
def get_site(url):
    response = requests.get(url)
    response.encoding = 'UTF-8'
    soup = BeautifulSoup(response.text , 'html.parser')
    
    return soup

In [4]:
def multithrd(DeptNo, CrsNo, depts):
    global threads 
    t = thrd.Thread(target = main, args = (DeptNo, CrsNo, dept))
    t.start()
    threads.update({(DeptNo, CrsNo) : t})

In [5]:
def main(DeptNo, CrsNo, dept):
    # set up a loop and wait for several seconds between each turn
    try:
        while True:
            course = Course_list(DeptNo, dept)
            print("{0:s}，餘額: {1:s}".format(course[CrsNo][0], course[CrsNo][1]))
            time.sleep(2)
    except KeyError:
        print("Course not found!\n")
        

In [6]:
def remove_dead_threads():
    global threads
    threads = {key : t for key, t in threads.items() if t.is_alive()}

In [ ]:
if __name__ == "__main__":
    #user input
    dept = Department_list()
    threads = {}
    while True:
        try:
            usr_DeptNo, usr_CrsNo = input('輸入系所及科目代碼(空白隔開 eg: A2 001)：').split()
            multithrd(usr_DeptNo, usr_CrsNo, dept)
            print(threads)
        except:
            print("Format error!\n")
        finally:
            remove_dead_threads()
       
        
            

輸入系所及科目代碼(空白隔開 eg: A2 001)：A9 001
{('A9', '001'): <Thread(Thread-6, started 6868)>}
藝術史與藝術批評，餘額: 額滿
輸入系所及科目代碼(空白隔開 eg: A2 001)：A9 000
{('A9', '001'): <Thread(Thread-6, started 6868)>, ('A9', '000'): <Thread(Thread-7, started 4812)>}
藝術史與藝術批評，餘額: 額滿
Course not found!

藝術史與藝術批評，餘額: 額滿
藝術史與藝術批評，餘額: 額滿
輸入系所及科目代碼(空白隔開 eg: A2 001)：A9 002
{('A9', '001'): <Thread(Thread-6, started 6868)>, ('A9', '000'): <Thread(Thread-7, stopped 4812)>, ('A9', '002'): <Thread(Thread-8, started 6796)>}
宗教哲學，餘額: 額滿
藝術史與藝術批評，餘額: 額滿
宗教哲學，餘額: 額滿
輸入系所及科目代碼(空白隔開 eg: A2 001)：A9 003
藝術史與藝術批評，餘額: 額滿
{('A9', '001'): <Thread(Thread-6, started 6868)>, ('A9', '002'): <Thread(Thread-8, started 6796)>, ('A9', '003'): <Thread(Thread-9, started 6328)>}
文本與歷史，餘額: 20
宗教哲學，餘額: 額滿
藝術史與藝術批評，餘額: 額滿
文本與歷史，餘額: 20
宗教哲學，餘額: 額滿
藝術史與藝術批評，餘額: 額滿
文本與歷史，餘額: 20
